## Import

In [19]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -p numpy,sklearn,pandas
%watermark -p ipywidgets,cv2,PIL,matplotlib,plotly,netron
%watermark -p torch,torchvision,torchaudio
# %watermark -p tensorflow,tensorboard,tflite
# %watermark -p onnx,tf2onnx,onnxruntime,tensorrt,tvm
# %matplotlib inline
# %config InlineBackend.figure_format='retina'
# %config IPCompleter.use_jedi = False

# %matplotlib inline
# %matplotlib widget
# from IPython.display import display, Markdown, HTML, IFrame, Image, Javascript
# from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
# display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

import sys, os, io, logging, time, random, math
import json, base64, requests, shutil
import argparse, shlex, signal
import numpy as np

import pandas as pd
import cv2
from glob import glob
import re
import yaml

np.set_printoptions(
    edgeitems=3, infstr='inf',
    linewidth=75, nanstr='nan', precision=6,
    suppress=True, threshold=100, formatter=None)

argparse.ArgumentParser.exit = lambda *arg, **kwargs: _IGNORE_

def _IMPORT(x, tag='main', debug=False):
    def __request_text(url):
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            raise RuntimeError(url)
    try:
        x = x.strip()
        if x[0] == '/' or x[1] == '/':
            with open(x) as fr:
                x = fr.read()
        elif 'github' in x or 'gitee' in x:
            if x.startswith('import '):
                x = x[7:]
            if x.startswith('https://'):
                x = x[8:]
            if not x.endswith('.py'):
                x = x + '.py'
            x = x.replace('blob/main/', '').replace('blob/master/', '')
            if x.startswith('raw.githubusercontent.com'):
                x = 'https://' + x
                x = __request_text(x)
            elif x.startswith('github.com'):
                x = x.replace('github.com', 'raw.githubusercontent.com')
                mod = x.split('/')
                x = 'https://' + '/'.join(mod[:3]) + f'/{tag}/' + '/'.join(mod[-3:])
                x = __request_text(x)
            elif x.startswith('gitee.com'):
                mod = x.split('/')
                x = 'https://' + '/'.join(mod[:3]) + f'/raw/{tag}/' + '/'.join(mod[3:])
                x = __request_text(x)
        if debug:
            return x
        else:
            exec(x, globals())
    except Exception as err:
        # sys.stderr.write(f'request {x} : {err}')
       pass

def _DIR(x, dumps=True, ret=True):
    attrs = sorted([y for y in dir(x) if not y.startswith('_')])
    result = '%s: %s' % (str(type(x))[8:-2], json.dumps(attrs) if dumps else attrs)
    if ret:
        return result
    print(result)


numpy  : 1.22.3
sklearn: 1.0.2
pandas : 1.4.1

ipywidgets: 7.7.1
cv2       : 4.8.1
PIL       : 9.0.1
matplotlib: 3.5.1
plotly    : 5.18.0
netron    : not installed

torch      : 1.12.0.dev20220327+cpu
torchvision: 0.13.0.dev20220327+cpu
torchaudio : 0.12.0.dev20220327+cpu



## Global 

In [142]:
TOP_DIR = '/jupyter/hzcsbet/gamebet'
DATASET_DIR = f'{TOP_DIR}/datasets/SoccerNet/tracking'

labels = ['ball', 'player', 'referee', 'goalkeepers']
label_dict = {'ball': 0, 'player': 1, 'referee': 2, 'goalkeeper': 3, 'goalkeepers': 3}

## Data Format Convert

In [132]:
yolo_base = f'{TOP_DIR}/datasets/yolov8'

#(1) image file path
yolo_train_img_dir = f'{yolo_base}/images/train'
yolo_valid_img_dir = f'{yolo_base}/images/valid'

#(2) label file path
yolo_train_label_dir = f'{yolo_base}/labels/train'
yolo_valid_label_dir = f'{yolo_base}/labels/valid'

#(3) config file path
yaml_file = f'{yolo_base}/data.yaml'

os.makedirs(yolo_train_img_dir, exist_ok=True)
os.makedirs(yolo_valid_img_dir, exist_ok=True)
os.makedirs(yolo_train_label_dir, exist_ok=True)
os.makedirs(yolo_valid_label_dir, exist_ok=True)

In [146]:
snmot_dirs = {}
snmot_dirs['test'] = sorted(glob(f'{DATASET_DIR}/test/SNMOT*'))
snmot_dirs['train'] = sorted(glob(f'{DATASET_DIR}/train/SNMOT*'))
t1 = snmot_dirs['test'][0]
print(t1)
!ls $t1

/jupyter/hzcsbet/gamebet/datasets/SoccerNet/tracking/test/SNMOT-116
det  gameinfo.ini  gt  img1  seqinfo.ini


In [64]:
!cat $t1/seqinfo.ini

[Sequence]
name=SNMOT-116
imDir=img1
frameRate=25
seqLength=750
imWidth=1920
imHeight=1080
imExt=.jpg


In [48]:
!cat $t1/gameinfo.ini

[Sequence]
name=SNMOT-116
gameID=7
actionPosition=975293
actionClass=Corner
visibility=visible
clipStart=969000
gameTimeStart=1 - 16:09
clipStop=999000
gameTimeStop=1 - 16:39
num_tracklets=27
trackletID_1= player team left;4
trackletID_2= player team left;93
trackletID_3= player team right;25
trackletID_4= referee;main
trackletID_5= player team left;A
trackletID_6= player team left;11
trackletID_7= player team right;34
trackletID_8= player team left;44
trackletID_9= player team right;33
trackletID_10= player team right;20
trackletID_11= player team right;10
trackletID_12= player team right;31
trackletID_13= player team right;14
trackletID_14= player team left;33
trackletID_15= player team left;8
trackletID_16= player team right;36
trackletID_17= player team left;36
trackletID_18= player team right;8
trackletID_19= goalkeepers team left;1
trackletID_20= ball;1
trackletID_21= player team right;B
trackletID_22= player team right;E
trackletID_23= player team left;C
trackletID_24= player te

In [51]:
!head -n 10 $t1/gt/gt.txt 

1,1,136,520,51,135,1,-1,-1,-1
2,1,138,521,50,134,1,-1,-1,-1
3,1,140,521,48,133,1,-1,-1,-1
4,1,142,521,47,132,1,-1,-1,-1
5,1,143,522,46,131,1,-1,-1,-1
6,1,146,523,44,129,1,-1,-1,-1
7,1,146,522,45,129,1,-1,-1,-1
8,1,147,522,45,130,1,-1,-1,-1
9,1,148,521,46,130,1,-1,-1,-1
10,1,149,521,47,131,1,-1,-1,-1


In [52]:
!tail -n 10 $t1/gt/gt.txt 

741,27,148,396,48,113,1,-1,-1,-1
742,27,140,397,49,112,1,-1,-1,-1
743,27,130,399,51,112,1,-1,-1,-1
744,27,125,400,50,112,1,-1,-1,-1
745,27,118,401,48,113,1,-1,-1,-1
746,27,110,403,46,113,1,-1,-1,-1
747,27,103,404,44,113,1,-1,-1,-1
748,27,94,405,42,114,1,-1,-1,-1
749,27,89,406,42,114,1,-1,-1,-1
750,27,80,409,42,114,1,-1,-1,-1


In [64]:
!cat $t1/seqinfo.ini

[Sequence]
name=SNMOT-116
imDir=img1
frameRate=25
seqLength=750
imWidth=1920
imHeight=1080
imExt=.jpg


In [169]:
# convert from x,y,w,h to yolo format
def get_yolo_format_bbox(img_w, img_h, box):
    w = int(box[2])
    h = int(box[3])
    xc = int(box[0]) + int(np.round(w/2))
    yc = int(box[1]) + int(np.round(h/2))
    box = [xc/img_w, yc/img_h, w/img_w, h/img_h]
    box = [f"{i:.4g}" for i in box]
    return box
    
# get SoccerNet label info 
def get_game_info(info):
    results = {}
    for line in open(info):
        m = re.match('trackletID_(\d+)= (\S*).*', line.replace(';', ' '))
        if m:
            if m.group(2) not in label_dict:
                continue 
            results[m.group(1)] = m.group(2)
    return results

def get_seq_info(info):
    results = {}
    for line in open(info):
        if line.startswith('name'):
            results['name'] = line.split('=')[1].strip()
        elif line.startswith('imDir'):
            results['img_dir'] = line.split('=')[1].strip()
        elif line.startswith('imWidth'):
            results['img_w'] = int(line.split('=')[1].strip())
        elif line.startswith('imHeight'):
            results['img_h'] = int(line.split('=')[1].strip())
        elif line.startswith('imExt'):
            results['img_ext'] = line.split('=')[1].strip()
    return results

In [120]:
seq_info = get_seq_info(f'{t1}/seqinfo.ini')
seq_info

{'name': 'SNMOT-116',
 'img_dir': 'img1',
 'img_w': 1920,
 'img_h': 1080,
 'img_ext': '.jpg'}

In [153]:
t1 = '/jupyter/hzcsbet/gamebet/datasets/SoccerNet/tracking/test/SNMOT-133'

In [166]:
game_info = get_game_info(f'{t1}/gameinfo.ini')
game_info

[Sequence]

[Sequence]

name=SNMOT-133

name=SNMOT-133

gameID=8

gameID=8

actionPosition=593638

actionPosition=593638

actionClass=Yellow card

actionClass=Yellow card

visibility=visible

visibility=visible

clipStart=569000

clipStart=569000

gameTimeStart=1 - 09:29

gameTimeStart=1 - 09:29

clipStop=599000

clipStop=599000

gameTimeStop=1 - 09:59

gameTimeStop=1 - 09:59

num_tracklets=29

num_tracklets=29

trackletID_1= player team left 36

trackletID_2= player team left 14

trackletID_3= player team left 33

trackletID_4= player team right 10

trackletID_5= player team right 14

trackletID_6= player team left a

trackletID_7= player team right b

trackletID_8= referee main

trackletID_9= player team left c

trackletID_10= player team right 7

trackletID_11= player team left 5

trackletID_12= player team left d

trackletID_13= referee side top

trackletID_14= ball 1

trackletID_15= player team right 26

trackletID_16= player team right x

trackletID_17= player team left 10

track

{'1': 'player',
 '2': 'player',
 '3': 'player',
 '4': 'player',
 '5': 'player',
 '6': 'player',
 '7': 'player',
 '8': 'referee',
 '9': 'player',
 '10': 'player',
 '11': 'player',
 '12': 'player',
 '13': 'referee',
 '14': 'ball',
 '15': 'player',
 '16': 'player',
 '17': 'player',
 '18': 'player',
 '19': 'player',
 '20': 'player',
 '21': 'player',
 '22': 'goalkeeper',
 '23': 'player',
 '24': 'player',
 '25': 'goalkeepers'}

In [170]:
results = []
for phase, dirs in snmot_dirs.items():
    if len(dirs) == 0:
        continue
    for mot in dirs:
        seq_info = get_seq_info(f'{mot}/seqinfo.ini')
        game_info = get_game_info(f'{mot}/gameinfo.ini')
        result = {}
        for line in open(f'{mot}/gt/gt.txt'):
            imgid, labid, *box = line.split(',')[:6]
            if labid in game_info:
                imgid = f'{int(imgid):06}'
                if imgid not in result:
                    result[imgid] = []
                box = get_yolo_format_bbox(seq_info['img_w'], seq_info['img_h'], box)
                result[imgid].append([label_dict[game_info[labid]], *box])
        seq_info['data'] = result
        results.append(seq_info)

In [171]:
len(results)

49

In [135]:
for item in results:
    for imgid, data in item['data'].items():
        imgfile = f'{item["name"]}_{imgid}{item["img_ext"]}'
        labfile = f'{item["name"]}_{imgid}.txt'
        img_src = f'{TEST_DATASET_DIR}/{item["name"]}/{item["img_dir"]}/{imgid}{item["img_ext"]}'
        img_dst = f'{yolo_valid_label_dir}/{imgfile}'
        if not os.path.exists(img_src):
            raise img_src
        print(img_dst)
        if not os.path.exists(img_dst):
            os.symlink(img_src, img_dst)
        print(imgfile, labfile)
        # with open(yolo_valid_label_dir, 'w') as fw:
        break

/jupyter/hzcsbet/gamebet/datasets/yolov8/labels/valid/SNMOT-116_000001.jpg
SNMOT-116_000001.jpg SNMOT-116_000001.txt


In [118]:
result['000001']

[[1, '0.08438', '0.5444', '0.02656', '0.125'],
 [1, '0.2625', '0.588', '0.02656', '0.1296'],
 [1, '0.3068', '0.537', '0.025', '0.1148'],
 [2, '0.2557', '0.4528', '0.02396', '0.1046'],
 [1, '0.299', '0.3185', '0.01771', '0.07778'],
 [1, '0.5026', '0.5722', '0.03073', '0.1296'],
 [1, '0.5286', '0.5639', '0.03594', '0.1269'],
 [1, '0.563', '0.5083', '0.02083', '0.113'],
 [1, '0.5875', '0.5074', '0.03333', '0.113'],
 [1, '0.6844', '0.475', '0.0276', '0.1046'],
 [1, '0.7307', '0.4509', '0.01667', '0.1019'],
 [1, '0.7484', '0.4417', '0.01927', '0.09722'],
 [1, '0.763', '0.4685', '0.03385', '0.09907'],
 [1, '0.7693', '0.4704', '0.01719', '0.1157'],
 [1, '0.7661', '0.463', '0.01406', '0.1009'],
 [1, '0.7776', '0.4713', '0.0151', '0.1102'],
 [1, '0.7865', '0.4676', '0.01198', '0.113'],
 [1, '0.7979', '0.4676', '0.02187', '0.1139'],
 [3, '0.8068', '0.4639', '0.02292', '0.1056'],
 [0, '0.25', '0.3583', '0.00625', '0.01019']]

## References

- [BoT-SORT: Robust Associations Multi-Pedestrian Tracking][5]
- [ByteTrack: Multi-Object Tracking by Associating Every Detection Box][1]
- [SoccerNet - Tracking][2]
- [Cool experiments at the intersection of Computer Vision and Sports][3]
- [Football players tracking — identifying players’ team based on their jersey colors using OpenCV][4]
- https://readmedium.com/chess-rolls-or-basketball-lets-create-a-custom-object-detection-model-ef53028eac7d
- https://medium.com/@amirhossein477/how-to-obtain-the-birds-eye-view-of-a-soccer-game-regardless-of-camera-angle-changes-90d627acd522
- https://medium.com/@amritangshu.mukherjee/tracking-football-players-with-yolov5-bytetrack-efa317c9aaa4
- https://www.jlburkhead.com

[5]: https://arxiv.org/pdf/2
[4]: https://towardsdatascience.com/football-players-tracking-identifying-players-team-based-on-their-jersey-colors-using-opencv-7eed1b8a1095
[3]: https://github.com/SkalskiP/sports
[2]: https://github.com/SoccerNet/sn-tracking
[1]: https://arxiv.org/pdf/2110.06864.pdf